# Data organization

In this notebook, the main information regarding Energy usage adapted from https://www.eia.gov/electricity/data.php was used.

In this specific project, the variable to analyze is the total generation per **energy source** (Solar, Wind, Thermal, among others) and per **producing sector** (Commercial, industrial, residential). However, to be able to use python resources like pandas and seaborn it is important to transform the information from the original .xlxs to .csv.

For this reason, in the following cells the file `generation_monthly.xlsx` was analyzed and transformed into a .csv that will be used for the Exploratory Data Analysis.

## Initial insights

As a first step, the pandas library was used for reading the main excel file obtained from https://www.eia.gov/electricity/data/state/generation_monthly.xlsx

In [1]:
import pandas as pd

After, with the use of `pandas.ExcelFile`, the data is saved in a variable for analyzing the conformation of the file and initial ways for data treatment

In [2]:
excel=pd.ExcelFile("data/generation_monthly.xlsx")

This new object stores all the relevant data of the spreadsheet by its different sheets, as shown by the property `.sheet_names`

In [3]:
#Extracting sheet names
excel.sheet_names

['2001_2002_FINAL',
 '2003_2004_FINAL',
 '2005-2007_FINAL',
 '2008-2009_FINAL',
 '2010-2011_FINAL',
 '2012_Final',
 '2013_Final',
 '2014_Final',
 '2015_Final',
 '2016_Final',
 '2017_Final',
 '2018_Final',
 '2019_Final',
 '2020_Final',
 '2021_Preliminary',
 '2022_Preliminary',
 'EnergySource_Notes']

As it was checked from the original File the first 5 sheets have a different format, so to extract the information and pass it to a dataframe in the next cell the sheet `2001_2002_FINAL`was parsed to a dataframe to see its columns and a first glance of data.

In [4]:
#Checking the formatting of the first 5 sheets
excel.parse('2001_2002_FINAL')

,YEAR,MONTH,STATE,TYPE OF PRODUCER,ENERGY SOURCE,GENERATION (Megawatthours)
0,2001,1,AK,Total Electric Power Industry,Coal,46903
1,2001,1,AK,Total Electric Power Industry,Petroleum,71085
2,2001,1,AK,Total Electric Power Industry,Natural Gas,367521
3,2001,1,AK,Total Electric Power Industry,Hydroelectric Conventional,104549
4,2001,1,AK,Total Electric Power Industry,Wind,87
...,...,...,...,...,...,...
41210,2002,12,US-TOTAL,"Combined Heat and Power, Industrial Power",Hydroelectric Conventional,529199
41211,2002,12,US-TOTAL,"Combined Heat and Power, Industrial Power",Wood and Wood Derived Fuels,2439123
41212,2002,12,US-TOTAL,"Combined Heat and Power, Industrial Power",Other Biomass,65609
41213,2002,12,US-TOTAL,"Combined Heat and Power, Industrial Power",Other,248231


As expected, this worksheet has the following columns:

1. Year
2. Month
3. State
4. Type of producer
5. Energy source
6. Generation

Then, the same was made with the `2012_Final` sheet as the rest of them follow the same structure, obtaining the following data frame:

In [5]:
excel.parse('2012_Final')

,"U.S. Department of Energy, The Energy Information Administration (EIA)",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,"Monthly Generation Data by State, Producer Sec...",NaN,NaN,NaN,NaN,NaN
1,Sources: EIA-923 Report,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,YEAR,MONTH,STATE,TYPE OF PRODUCER,ENERGY SOURCE,GENERATION (Megawatthours)
4,2012,1,AK,Total Electric Power Industry,Total,724707.28
...,...,...,...,...,...,...
23612,2012,12,WY,"Combined Heat and Power, Industrial Power",Coal,41408.02
23613,2012,12,WY,"Combined Heat and Power, Industrial Power",Natural Gas,47514.65
23614,2012,12,WY,"Combined Heat and Power, Industrial Power",Other,5614.57
23615,2012,12,WY,"Combined Heat and Power, Industrial Power",Other Gases,29254.19


In this case, as there is additional text on the first three rows, to create the correct dataframe the property header was used for starting on the 4th row and obtaining the following data

In [6]:
excel.parse('2012_Final',header=4)

,YEAR,MONTH,STATE,TYPE OF PRODUCER,ENERGY SOURCE,GENERATION (Megawatthours)
0,2012,1,AK,Total Electric Power Industry,Total,724707.28
1,2012,1,AK,Total Electric Power Industry,Coal,59799.16
2,2012,1,AK,Total Electric Power Industry,Hydroelectric Conventional,162651.88
3,2012,1,AK,Total Electric Power Industry,Natural Gas,385256.38
4,2012,1,AK,Total Electric Power Industry,Other,0.00
...,...,...,...,...,...,...
23608,2012,12,WY,"Combined Heat and Power, Industrial Power",Coal,41408.02
23609,2012,12,WY,"Combined Heat and Power, Industrial Power",Natural Gas,47514.65
23610,2012,12,WY,"Combined Heat and Power, Industrial Power",Other,5614.57
23611,2012,12,WY,"Combined Heat and Power, Industrial Power",Other Gases,29254.19


The main structure is the same, but in the case of the generation the column name comes in different formatting as with the columns analysis there is a /n character. In this case, this needs to be solved to be able to append all the relevant data in a single file.

## Final processing

For linking all the sheets a for loop through all the sheets was used, whereas in an empty data frame the information is appended. In the first 5 elements, the parser is used and with the following ones, the column data of generation is replaced for "GENERATION (Megawatthours)" before updating this data frame.

In [7]:
#Process to append all the tables in a final dataframe
number=0
df=pd.DataFrame()
for i in excel.sheet_names[:-1]:
    number=number+1
    if number > 5: #Latest sheets
        pre_df=excel.parse(i,header=4)
        pre_df.rename(columns={"GENERATION\n(Megawatthours)":"GENERATION (Megawatthours)"},inplace=True)
        df=df.append(pre_df)
    else:
        df=df.append(excel.parse(i))

After this code is executed the following dataframe is obtained

In [8]:
df

,YEAR,MONTH,STATE,TYPE OF PRODUCER,ENERGY SOURCE,GENERATION (Megawatthours)
0,2001,1,AK,Total Electric Power Industry,Coal,46903.0
1,2001,1,AK,Total Electric Power Industry,Petroleum,71085.0
2,2001,1,AK,Total Electric Power Industry,Natural Gas,367521.0
3,2001,1,AK,Total Electric Power Industry,Hydroelectric Conventional,104549.0
4,2001,1,AK,Total Electric Power Industry,Wind,87.0
...,...,...,...,...,...,...
10581,2022,5,WY,"Electric Generators, Electric Utilities",Coal,2071403.0
10582,2022,5,WY,"Electric Generators, Electric Utilities",Hydroelectric Conventional,96790.0
10583,2022,5,WY,"Electric Generators, Electric Utilities",Natural Gas,91570.0
10584,2022,5,WY,"Electric Generators, Electric Utilities",Petroleum,1812.0


Finally, for checking that all the columns have the expected values as presented in https://www.eia.gov/electricity/monthly/ the df dataframe is checked:

In [9]:
for i in df.columns[:-1]:
    print(i + ": ",  df[i].unique(), ", Total Values:", len(df[i].unique()))

YEAR:  [2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014
 2015 2016 2017 2018 2019 2020 2021 2022] , Total Values: 22
MONTH:  [ 1  2  3  4  5  6  7  8  9 10 11 12] , Total Values: 12
STATE:  ['AK' 'AL' 'AR' 'AZ' 'CA' 'CO' 'CT' 'DC' 'DE' 'FL' 'GA' 'HI' 'IA' 'ID'
 'IL' 'IN' 'KS' 'KY' 'LA' 'MA' 'MD' 'ME' 'MI' 'MN' 'MO' 'MS' 'MT' 'NC'
 'ND' 'NE' 'NH' 'NJ' 'NM' 'NV' 'NY' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD'
 'TN' 'TX' 'UT' 'VA' 'VT' 'WA' 'WI' 'WV' 'WY' 'US-TOTAL' 'US-Total'] , Total Values: 53
TYPE OF PRODUCER:  ['Total Electric Power Industry' 'Electric Generators, Electric Utilities'
 'Combined Heat and Power, Electric Power'
 'Combined Heat and Power, Commercial Power'
 'Combined Heat and Power, Industrial Power'
 'Electric Generators, Independent Power Producers'] , Total Values: 6
ENERGY SOURCE:  ['Coal' 'Petroleum' 'Natural Gas' 'Hydroelectric Conventional' 'Wind'
 'Total' 'Other Gases' 'Nuclear' 'Wood and Wood Derived Fuels'
 'Other Biomass' 'Other' 'Solar Thermal 

As expected, the fields coincide but in the case of the STATE column two values represent the total consumption of the USA, which are `"US-TOTAL"` and `"US-Total"`. For correcting this, the string upper method was used.

In [10]:
df["STATE"]=df["STATE"].str.upper()

Then, the resulting dataframe is saved in a CSV to be used for further analysis

In [11]:
df.to_csv("data/organised_Gen.csv")

## Final insights

With this processing and with the aid of US Energy Information Administration documentation now there is a file available for further analysis and eventual Machine Learning and clustering models.